In [2]:
from pylyra import *

# -- Metrics --------

print("Definindo metrica")
t, r, th, vph = sp.symbols('t r theta varphi', real=True)
x = (t, r, th, vph)
dim = 4

f1 = sp.Function('f1')(r)
f2 = sp.Function('f2')(r)

g = sp.diag(sp.exp(f1), -sp.exp(f2), -r**2, -r**2*sp.sin(th)**2)  # g_{mu nu}

# -- SpaceTime ----------
st = SpaceTime(dim=4, coords=x, metric=g)

# -- Escala de Lyra ----------
print("Definindo escala")
phi = st.set_scale(sp.Function('phi')(r))

# -- Tensor de Levi Civita ----------
print("Definindo LeviCivita")
eps = st.tensor.from_function(
    lambda mu, nu, rho, sig: sp.sqrt(-st.detg) * sp.LeviCivita(mu, nu, rho, sig),
    signature=(d, d, d, d),
    name="eps",
)

# -- Non-Metricity ----------
print("Definindo Non-Metricity")
M = st.tensor.from_function(
    lambda a, b, c: 0,
    signature=(u, d, d),
    name="M",
)
st.set_nonmetricity(M)

# -- Torsion ----------
print("Definindo Torsion")

def torsion_axial(mu):
    if mu == 0:
        return sp.Function('s_t')(r)
    if mu == 1:
        return sp.Function('s_r')(r)
    return 0

s = st.tensor.from_function(torsion_axial, signature=(u,), name="s")


def torsion_trace(mu):
    if mu == 0:
        return sp.Function('v_t')(r)
    if mu == 1:
        return sp.Function('v_r')(r)
    return 0

v = st.tensor.from_function(torsion_trace, signature=(d,), name="v")


def torsion_component(m, n, r_):
    return (
        sum(eps(d, d, d, d).comp[m, n, r_, sig] * s(u).comp[sig] for sig in range(dim))
        + sp.Rational(1, 3) * (
            st.g[m, n] * v(d).comp[r_]
            - st.g[m, r_] * v(d).comp[n]
        )
    )

tau = st.tensor.from_function(torsion_component, signature=(d, d, d), name="tau")
st.set_torsion(tau)

# -- Curvature ----------
print("Definindo Curvature")

def curvature_element(l, a, m, n):
    Gamma = st.gamma.components
    return (
        1 / (phi**2) * sp.diff(phi * Gamma[l, a, n], x[m])
        - 1 / (phi**2) * sp.diff(phi * Gamma[l, a, m], x[n])
        + sum(Gamma[r, a, n] * Gamma[l, r, m] for r in range(dim))
        - sum(Gamma[r, a, m] * Gamma[l, r, n] for r in range(dim))
    )

Curv = st.tensor.from_function(curvature_element, signature=(u, d, d, d), name="R")

# -- Ricci ----------
print("Definindo Ricci")

def ricci_element(a, m):
    return sp.expand(sp.simplify(
        sum(Curv(u, d, d, d).comp[l, a, m, l] for l in range(dim))
    ))

Ricc = st.tensor.from_function(ricci_element, signature=(d, d), name="Ric")

# -- Curvature Scalar ----------
print("Definindo Curvature Scalar")
R = sp.expand(sum(Ricc(u, d).comp[l, l] for l in range(dim))).subs(th, sp.pi/2)

# -- Contracoes com nova sintaxe ----------
_, a, b, c = st.index("empty a b c")

R_scalar_1 = st.contract(Ricc.idx((a, _), (_, a)))


Definindo metrica


KeyboardInterrupt: 

In [9]:
Ricc(u, u).comp[1,1]

(s_t(r)**2*exp(f1(r))*exp(f2(r))/2 + v_t(r)**2*exp(-f1(r))*exp(f2(r))/9 + v_r(r)*Derivative(f1(r), r)/(6*phi(r)) + Derivative(f1(r), r)**2/(4*phi(r)**2) - Derivative(f1(r), r)*Derivative(f2(r), r)/(4*phi(r)**2) + Derivative(f1(r), (r, 2))/(2*phi(r)**2) + Derivative(f1(r), r)*Derivative(phi(r), r)/(2*phi(r)**3) + 2*v_r(r)/(3*r*phi(r)) - Derivative(f2(r), r)/(r*phi(r)**2) + 2*Derivative(phi(r), r)/(r*phi(r)**3))*exp(-2*f2(r))

In [10]:
st.contract(Ricc.idx((a, _), (_, a)))

In [12]:
R_scalar_1.expr

-(s_t(r)**2*exp(f1(r))*exp(f2(r))/2 + v_t(r)**2*exp(-f1(r))*exp(f2(r))/9 + v_r(r)*Derivative(f1(r), r)/(6*phi(r)) + Derivative(f1(r), r)**2/(4*phi(r)**2) - Derivative(f1(r), r)*Derivative(f2(r), r)/(4*phi(r)**2) + Derivative(f1(r), (r, 2))/(2*phi(r)**2) + Derivative(f1(r), r)*Derivative(phi(r), r)/(2*phi(r)**3) + 2*v_r(r)/(3*r*phi(r)) - Derivative(f2(r), r)/(r*phi(r)**2) + 2*Derivative(phi(r), r)/(r*phi(r)**3))*exp(-f2(r)) + (s_r(r)**2*exp(f1(r))*exp(f2(r))/2 + v_r(r)**2*exp(f1(r))*exp(-f2(r))/9 + v_r(r)*exp(f1(r))*exp(-f2(r))*Derivative(f2(r), r)/(6*phi(r)) - exp(f1(r))*exp(-f2(r))*Derivative(v_r(r), r)/(3*phi(r)) + v_r(r)*exp(f1(r))*exp(-f2(r))*Derivative(phi(r), r)/(3*phi(r)**2) - exp(f1(r))*exp(-f2(r))*Derivative(f1(r), r)**2/(4*phi(r)**2) + exp(f1(r))*exp(-f2(r))*Derivative(f1(r), r)*Derivative(f2(r), r)/(4*phi(r)**2) - exp(f1(r))*exp(-f2(r))*Derivative(f1(r), (r, 2))/(2*phi(r)**2) + exp(f1(r))*exp(-f2(r))*Derivative(f2(r), r)*Derivative(phi(r), r)/(2*phi(r)**3) - exp(f1(r))*exp(-

In [2]:
from pylyra import *
# supondo que space já tenha coords e connection (Gamma) definidos
T = space.from_function(lambda a, b: sp.Function("T_%s%s" % (a,b))(*space.coords),
                        signature=(u, d))

# adiciona o índice covariante no fim: (u, d, d)
dT = T.nabla(deriv_position="append")

# componente: dT[a,b,k] = ∇_k T^a_b
dT.comp[0, 1, 2]


r**2*T_21(t, r, theta, varphi)*v_t(r)*exp(-f1(r))/3 + r**2*T_31(t, r, theta, varphi)*s_r(r)*sqrt(exp(f1(r) + f2(r)))*exp(-f1(r))*Abs(sin(theta))/2 - T_02(t, r, theta, varphi)*v_r(r)/3 - T_02(t, r, theta, varphi)*Derivative(phi(r), r)/phi(r)**2 + T_03(t, r, theta, varphi)*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))/(2*sin(theta)**2) + Derivative(T_01(t, r, theta, varphi), theta) - T_02(t, r, theta, varphi)/(r*phi(r))

In [8]:
st.gamma[0,0,0]

ValueError: Conexao nao definida.

# Sandbox

In [1]:
from pylyra import *

# -- Metrics --------

print("Definindo metrica")
t, r, th, vph = sp.symbols('t r theta varphi', real=True)
x = (t, r, th, vph)
dim = 4

lamb = sp.Function('lambda')
nu = sp.Function('nu')

g = sp.diag(sp.exp(lamb(r)), -sp.exp(nu(r)), -r**2, -r**2*sp.sin(th)**2)  # g_{mu nu}

# -- SpaceTime ----------
st = SpaceTime(dim=4, coords=x, metric=g)


# -- Curvature ----------
print("Definindo Curvature")

def curvature_element(l, a, m, n):
    Gamma = st.gamma.components
    return (
        1 / (st.phi**2) * sp.diff(st.phi * Gamma[l, a, n], x[m])
        - 1 / (st.phi**2) * sp.diff(st.phi * Gamma[l, a, m], x[n])
        + sum(Gamma[r, a, n] * Gamma[l, r, m] for r in range(dim))
        - sum(Gamma[r, a, m] * Gamma[l, r, n] for r in range(dim))
    )

Curv = st.tensor.from_function(curvature_element, signature=(u, d, d, d), name="R")

# -- Ricci ----------
print("Definindo Ricci")

def ricci_element(a, m):
    return sp.expand(sp.simplify(
        sum(Curv(u, d, d, d).comp[l, a, m, l] for l in range(dim))
    ))

Ricc = st.tensor.from_function(ricci_element, signature=(d, d), name="Ric")

# -- Curvature Scalar ----------
print("Definindo Curvature Scalar")
R = sp.expand(sum(Ricc(u, d).comp[l, l] for l in range(dim))).subs(th, sp.pi/2)

# -- Contracoes com nova sintaxe ----------
_, a, b, c = st.index("empty a b c")

R_scalar_1 = st.contract(Ricc.idx((a, _), (_, a)))


Definindo metrica
Definindo Curvature
Definindo Ricci
Definindo Curvature Scalar


In [8]:
Ricc(d, d).comp[0, 0] - sp.Rational(1,2) * st.g(d, d).comp[0, 0] * R_scalar_1

TypeError: 'ImmutableDenseNDimArray' object is not callable

In [3]:
R_scalar_1

In [5]:
st.g

[[exp(lambda(r)), 0, 0, 0], [0, -exp(nu(r)), 0, 0], [0, 0, -r**2, 0], [0, 0, 0, -r**2*sin(theta)**2]]

In [ ]:
print()


x = (t, r, th, vph) = sp.symbols('t r theta varphi', real=True)

st = SpaceTime(
    coords=x,
    metric=sp.diag(sp.exp(sp.Function('f1')(r)), -sp.exp(f2 = sp.Function('f2')(r)), -r**2, -r**2*sp.sin(th)**2),
)

# nesse ponto, phi=1, tau=0, M=0


# lembre da classe builtn Tensor. Ela deve receber uma métrica e conexão e escala no cconstrutor. Tensor deve ter informações sobre: nome (id unico), label (expressao para identificar o tensor, R, T, T_i) rank


# st deve ter um atributo interno .connection ou .gamma. Crie uma classe interna para abstrair isso


st.set_scale(sp.Function('phi')(r))

st.scale   #-> isso deve retornar uma instancia de Tensor de rank 0


st.set_torsion(..) # adicona a torção

st.set_non_metricity(...)


st.tensor.from_function(...)
st.tensor.generic(...)
st.tensor.generic(...)


# Algo parecido com isso seria legal


A = st.tensor.from_function( ..., signature=(d,d)) 
B = st.tensor.from_function( ..., signature=(d,d, d)) 


_,a,b,c = st.index("empty a b c")

st.contract(A.idx(up=(a,_),down=(.,a)), indexes=(a,)]  #-> reconhece automaticamente os indices repetidos; o resultado deve ser um tensor de rank 0 

st.contract(A.idx(up=(a,b),down=(_,_)), B.idx(up=(_,_,_),down=(a,b,c))] #-> reconhece automaticamente os indices repetidos; o resultado deve ser um tensor de rank 1


st.eval_contract("A^{a,}_{ ,a}")    #-> reconhece automaticamente os indices repetidos; o resultado deve ser um tensor de rank 0 
st.eval_contract("A^{a,b} B_{a,b,c}") #-> reconhece automaticamente os indices repetidos; o resultado deve ser um tensor de rank 1
            


Na verdade, queria melhorar as abstrações. Da forma como está agora:

- A métrica não é um tensor
- Escalares não são tensores
- Ter alguma forma de fazer contrações


Pensei em ter um elemento que abstraia o espaço-tempo. Manifold? Não sei

Dai, deveriamos ter métodos set_scale. Se não passar, usa-se phi = sp.Function('phi')(r)

um método set metric
set torsion
set metrica compatibility

e uma forma de fazer contração de indices


In [ ]:
M.contract(
    
)

In [1]:
from pylyra import *

t, r, th, vph = sp.symbols('t r theta varphi', real=True)
x = (t, r, th, vph)

st = SpaceTime(
    dim=4,
    coords=x,
    metric=sp.diag(
        sp.exp(sp.Function('f1')(r)),
        -sp.exp(sp.Function('f2')(r)),
        -r**2,
        -r**2*sp.sin(th)**2
    ),
)

# defaults
st.scale      # Tensor rank 0 (phi = 1)
st.torsion    # tau = 0
st.nonmetricity  # M = 0

st.set_scale(sp.Function('phi')(r))

A = st.tensor.from_function(lambda i, j: sp.Function('A')(r), signature=(d, d), name="A")
B = st.tensor.from_function(lambda i, j, k: sp.Function('B')(r), signature=(d, d, d), name="B")

_, a, b, c = st.index("empty a b c")

# índices por posição (up/down por posição do rank)
res1 = st.contract(A.idx(up=(a, _), down=(_, a)))

# contração entre dois tensores
res2 = st.contract(
    A.idx(up=(a, b), down=(_, _)),
    B.idx(up=(_, _, _), down=(a, b, c))
)

# # parsing por string
# res3 = st.eval_contract("A^{a,}_{ ,a}")
# res4 = st.eval_contract("A^{a,b} B_{a,b,c}")


In [2]:
st.scale

AttributeError: 'ImmutableDenseNDimArray' object has no attribute '_repr_html_'